In [1]:
import geemap
import ee
geemap.set_proxy(port=10809)
ee.Authenticate()
ee.Initialize(project='ee-mrwurenzhe')

In [ ]:
# 定义时空范围
start_date = '2023-01-01'
end_date = '2023-12-31'
region = ee.Geometry.Rectangle([70, 10, 140, 60])  # 东亚范围（min_lon, min_lat, max_lon, max_lat）

# 加载ERA5月均数据集
era5_monthly = ee.ImageCollection('ECMWF/ERA5/MONTHLY') \
    .filterDate(start_date, end_date) \
    .select(['u_component_of_wind_10m', 'v_component_of_wind_10m'])

# 计算时间平均（所有月份的平均）
u_mean = era5_monthly.select('u_component_of_wind_10m').mean()
v_mean = era5_monthly.select('v_component_of_wind_10m').mean()

# 将u/v合并为单张图像（方便导出）
mean_wind = u_mean.addBands(v_mean)

# 可视化参数
vis_params = {
    'bands': ['u_component_of_wind_10m', 'v_component_of_wind_10m'],
    'min': -10,  # 风速范围（m/s）
    'max': 10,
    'palette': ['blue', 'white', 'red']  # u/v分量的颜色
}

# 在地图中显示
Map = geemap.Map()
Map.centerObject(region, 4)
Map.addLayer(mean_wind, vis_params, 'ERA5 Mean Wind (2023)')
Map.addLayer(region, {'color': 'yellow'}, 'ROI')
Map